In [424]:
import pydeck as pdk
import json

testjson = {
  "type": "Feature",
  "geometry": {
    "type": "Polygon",
    "coordinates": [48.2024752, 16.2784412]
  },
  "properties": {
    "name": "Dinagat Islands",
    "height": 10000000000
  }
}
testjson



{'type': 'Feature',
 'geometry': {'type': 'Polygon', 'coordinates': [48.2024752, 16.2784412]},
 'properties': {'name': 'Dinagat Islands', 'height': 10000000000}}

In [425]:
LAND_COVER = [[[16.3302836, 48.2018468], [ 16.369897,48.209112], 
               [16.374875,48.192750 ], [16.333161,48.1893172]]]

polygon_layer = pdk.Layer(
    "PolygonLayer",
    LAND_COVER,
    stroked=False,
    extruded=True,
    # processes the data as a flat longitude-latitude pair
    get_polygon="-",
    get_fill_color=[200, 0, 0, 255],
    get_elevation = "height"

)
# Set the initial view state (adjust as needed)
view_state = pdk.ViewState(
    latitude=48.2024752,
    longitude=16.2784412,
    zoom=12,
)# Create a Pydeck deck
deck = pdk.Deck(polygon_layer, initial_view_state=view_state)
deck

{
  "initialViewState": {
    "latitude": 48.2024752,
    "longitude": 16.2784412,
    "zoom": 12
  },
  "layers": [
    {
      "@@type": "PolygonLayer",
      "data": [
        [
          [
            16.3302836,
            48.2018468
          ],
          [
            16.369897,
            48.209112
          ],
          [
            16.374875,
            48.19275
          ],
          [
            16.333161,
            48.1893172
          ]
        ]
      ],
      "extruded": true,
      "getElevation": "@@=height",
      "getFillColor": [
        200,
        0,
        0,
        255
      ],
      "getPolygon": "@@=-",
      "id": "5454e001-caa6-4f1a-8073-e8db3f624253",
      "stroked": false
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}

In [426]:
from owslib.wfs import WebFeatureService

# Define the URL of the WFS service 
#https://data.wien.gv.at/daten/geo?version=1.0.0&typeName=ogdwien:FMZKBKMOGD&outputFormat=shape-zip&SRS=EPSG:31256&BBOX=2748,341794,4516,
wfs_url = 'https://data.wien.gv.at/daten/geo?'

# Connect to the WFS service
wfs = WebFeatureService(wfs_url, version='1.1.0')

In [427]:
layer_name = 'ogdwien:SOLARANLAGLSTGOGD'
layer = wfs[layer_name]
layer.__dict__

{'auth': <Authentication shared=False username=None password=None cert=None verify=True auth_delegate=None>,
 'headers': None,
 'id': 'ogdwien:SOLARANLAGLSTGOGD',
 'title': 'Solarpotential 2022 Leistung',
 'abstract': None,
 'keywords': ['features', 'V_OGD_SOLARPOT2022_LEISTG_F'],
 'boundingBoxWGS84': (16.187039178691105,
  48.120313294392396,
  16.577143629025443,
  48.31874678043532),
 'crsOptions': [urn:x-ogc:def:crs:EPSG::31256],
 'verbOptions': [],
 'outputFormats': [],
 'metadataUrls': [],
 'styles': None,
 'timepositions': None,
 'defaulttimeposition': None}

In [428]:
wfs.get_schema(layer_name)

{'properties': {'OBJECTID': 'decimal',
  'STR': 'string',
  'ONR': 'string',
  'ADRESSE': 'string',
  'BEZ': 'string',
  'YR': 'decimal',
  'SCHLECHT': 'decimal',
  'MITTEL': 'decimal',
  'GUT': 'decimal',
  'SEHRGUT': 'decimal',
  'ANLAGENLEISTUNG': 'decimal'},
 'required': ['OBJECTID'],
 'geometry': 'GeometryCollection',
 'geometry_column': 'SHAPE'}

In [429]:
import json


response = wfs.getfeature(
    typename=layer_name,
    #bbox= (16.187039178691105,
  #48.120313294392396,
 # 16.577143629025443,
  #48.31874678043532),  # Vienna bounding box
    srsname='urn:x-ogc:def:crs:EPSG:31256',
    maxfeatures=10,
    outputFormat="JSON",
    )



In [430]:

json_features = response.read()
json_features

b'{"type":"FeatureCollection","features":[{"type":"Feature","id":"SOLARANLAGLSTGOGD.35","geometry":{"type":"Polygon","coordinates":[[[-2145.891,338315.798],[-2147.44,338316.34],[-2155.57,338319.22],[-2155.65,338318.98],[-2156.27,338317.12],[-2156.59,338317.24],[-2157.429,338314.813],[-2160.34,338306.39],[-2151.53,338303.31],[-2151.365,338303.828],[-2148.164,338313.846],[-2148.07,338314.14],[-2148.19,338314.19],[-2147.988,338314.746],[-2147.67,338315.62],[-2144.93,338314.62],[-2144.67,338315.37],[-2145.891,338315.798]]]},"geometry_name":"SHAPE","properties":{"OBJECTID":35,"STR":" ","ONR":" ","ADRESSE":" ","BEZ":" ","YR":1122.2840651,"SCHLECHT":1,"MITTEL":19,"GUT":48,"SEHRGUT":21,"ANLAGENLEISTUNG":4}},{"type":"Feature","id":"SOLARANLAGLSTGOGD.36","geometry":{"type":"Polygon","coordinates":[[[-4637.9,340351.01],[-4640.96,340351.03],[-4640.94,340353.83],[-4653.3,340353.93],[-4653.41,340340.18],[-4642.57,340340.092],[-4641.319,340340.082],[-4641.06,340340.08],[-4641.01,340344.82],[-4638.88,

In [431]:
feature_collection = json.loads(json_features)
feature_collection

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'SOLARANLAGLSTGOGD.35',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-2145.891, 338315.798],
      [-2147.44, 338316.34],
      [-2155.57, 338319.22],
      [-2155.65, 338318.98],
      [-2156.27, 338317.12],
      [-2156.59, 338317.24],
      [-2157.429, 338314.813],
      [-2160.34, 338306.39],
      [-2151.53, 338303.31],
      [-2151.365, 338303.828],
      [-2148.164, 338313.846],
      [-2148.07, 338314.14],
      [-2148.19, 338314.19],
      [-2147.988, 338314.746],
      [-2147.67, 338315.62],
      [-2144.93, 338314.62],
      [-2144.67, 338315.37],
      [-2145.891, 338315.798]]]},
   'geometry_name': 'SHAPE',
   'properties': {'OBJECTID': 35,
    'STR': ' ',
    'ONR': ' ',
    'ADRESSE': ' ',
    'BEZ': ' ',
    'YR': 1122.2840651,
    'SCHLECHT': 1,
    'MITTEL': 19,
    'GUT': 48,
    'SEHRGUT': 21,
    'ANLAGENLEISTUNG': 4}},
  {'type': 'Feature',
   'id': 'SOLARANLAGLSTGOGD.36',
   'geo

In [432]:
features = feature_collection["features"]
for feature in features:
    print(feature['properties'])

{'OBJECTID': 35, 'STR': ' ', 'ONR': ' ', 'ADRESSE': ' ', 'BEZ': ' ', 'YR': 1122.2840651, 'SCHLECHT': 1, 'MITTEL': 19, 'GUT': 48, 'SEHRGUT': 21, 'ANLAGENLEISTUNG': 4}
{'OBJECTID': 36, 'STR': 'Minorgasse', 'ONR': '74', 'ADRESSE': '14., Minorgasse 74', 'BEZ': '1140', 'YR': 1041.6187741, 'SCHLECHT': 15, 'MITTEL': 38, 'GUT': 38, 'SEHRGUT': 0, 'ANLAGENLEISTUNG': 2}
{'OBJECTID': 37, 'STR': 'Stiegengasse', 'ONR': '6', 'ADRESSE': '6., Stiegengasse 6', 'BEZ': '1060', 'YR': 1089.81610074, 'SCHLECHT': 25, 'MITTEL': 112, 'GUT': 208, 'SEHRGUT': 36, 'ANLAGENLEISTUNG': 15}
{'OBJECTID': 38, 'STR': 'Tewelegasse', 'ONR': '10', 'ADRESSE': '13., Tewelegasse 10', 'BEZ': '1130', 'YR': 1177.5480808, 'SCHLECHT': 0, 'MITTEL': 1, 'GUT': 4, 'SEHRGUT': 16, 'ANLAGENLEISTUNG': 1}
{'OBJECTID': 39, 'STR': 'Gablenzgasse', 'ONR': '26', 'ADRESSE': '16., Gablenzgasse 26', 'BEZ': '1160', 'YR': 1164.3135829, 'SCHLECHT': 4, 'MITTEL': 49, 'GUT': 140, 'SEHRGUT': 157, 'ANLAGENLEISTUNG': 19}
{'OBJECTID': 40, 'STR': 'Gallitzinstr

In [433]:
features = [feature for feature in features if feature['properties'].get('STR') != ' ' and feature['properties'].get('ADRESSE') != ' ']

In [434]:
formatted_features = []
addressbuch = dict()
addressbuch2 = list()

for index, feature in enumerate(features):
    geometry_type = feature['geometry']['type']
    coordinates = feature['geometry']['coordinates']
    properties = feature['properties']
    adresse = feature['properties']['ADRESSE']
    leistung = feature['properties']['YR']
    anlagenleistung = feature['properties']['ANLAGENLEISTUNG']

    if geometry_type == 'Polygon':
        print(f"Adresse: {adresse}")
        print(f"Polygon: Coordinates: {coordinates}")

    addressbuch[index] = {
        "Adresse": adresse,
        "Leistung": leistung,
        "Anlagenleistung": anlagenleistung,
        # Add other properties as needed
    }

    addressbuch2.append(adresse)

# Now addressbuch has numerical indices as keys (0, 1, 2, ...)
# and each index maps to a dictionary containing address information.

user_address = input("Bitte gib deine Adresse ein (Bezirk, Straße Nr):")
user_address_parts = user_address.split(', ')
user_bezirk, user_strasse_nr = user_address_parts[0], user_address_parts[1]
user_key = f"{user_bezirk}, {user_strasse_nr}"

if user_key in addressbuch2:
    # Get the index for the given user address
    index_for_user_address = addressbuch2.index(user_key)
    print(addressbuch[index_for_user_address])
else:
    print("Address not found. Please try again.")

Adresse: 14., Minorgasse 74
Polygon: Coordinates: [[[-4637.9, 340351.01], [-4640.96, 340351.03], [-4640.94, 340353.83], [-4653.3, 340353.93], [-4653.41, 340340.18], [-4642.57, 340340.092], [-4641.319, 340340.082], [-4641.06, 340340.08], [-4641.01, 340344.82], [-4638.88, 340344.813], [-4638.63, 340344.812], [-4637.95, 340344.81], [-4637.9, 340351.01]]]
Adresse: 6., Stiegengasse 6
Polygon: Coordinates: [[[1949.9, 339933.85], [1946.71, 339938.5], [1945.35, 339937.57], [1941.389, 339943.341], [1938.54, 339947.49], [1937.72, 339946.93], [1932.036, 339943.341], [1928.258, 339940.955], [1928.437, 339940.666], [1928.703, 339940.845], [1930.19, 339938.638], [1929.971, 339938.479], [1935.453, 339930.485], [1936.426, 339929.061], [1937.163, 339927.982], [1939.497, 339924.566], [1941.151, 339922.132], [1942.661, 339919.911], [1942.848, 339919.636], [1942.905, 339919.669], [1944.167, 339920.523], [1952.96, 339926.47], [1948.54, 339932.92], [1949.9, 339933.85]]]
Adresse: 13., Tewelegasse 10
Polygon:

{'Adresse': '14., Minorgasse 74', 'Leistung': 1041.6187741, 'Anlagenleistung': 2}


In [435]:
testadresse = "Wien, " + user_address
testadresse

'Wien, 14., Minorgasse 74'

In [436]:
index_to_access = index_for_user_address
specific_feature = features[index_to_access]
print(specific_feature["geometry"]["coordinates"])

[[[-4637.9, 340351.01], [-4640.96, 340351.03], [-4640.94, 340353.83], [-4653.3, 340353.93], [-4653.41, 340340.18], [-4642.57, 340340.092], [-4641.319, 340340.082], [-4641.06, 340340.08], [-4641.01, 340344.82], [-4638.88, 340344.813], [-4638.63, 340344.812], [-4637.95, 340344.81], [-4637.9, 340351.01]]]


In [437]:
from geopy.geocoders import Nominatim

# Replace 'YourAPIKey' with your actual API key if you're using a service that requires one.
geolocator = Nominatim(user_agent="my_geocoder_app")

def get_lat_long_from_address(address):
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None

# Example usage:
address = testadresse
coordinates_adress = get_lat_long_from_address(address)

if coordinates_adress:
    print(f"Latitude: {coordinates_adress[0]}, Longitude: {coordinates_adress[1]}")
else:
    print("Could not retrieve coordinates for the given address.")

Latitude: 48.2024752, Longitude: 16.2784412


In [438]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:31256", "EPSG:4326", always_xy=True)

In [439]:
def transform_coordinates(coords):
    return list(transformer.transform(*coords))

def transform_feature(feature):
    coords = feature["geometry"]["coordinates"][index_to_access]
    feature["geometry"]["coordinates"] = [[transform_coordinates(coord) for coord in coords]]



In [440]:
n = 0
for i, feature in enumerate(features):
    transform_feature(feature)
        
print(feature["geometry"]["coordinates"])

[[[16.240122226056116, 48.207353709473296], [16.240129138780848, 48.20736237571967], [16.24016444065683, 48.20740665184974], [16.240196617714727, 48.20739516737888], [16.240219033512133, 48.207423424801625], [16.24018685644153, 48.207434909278696], [16.24018025950788, 48.207437242011125], [16.240193816857886, 48.207454070652], [16.240126971922976, 48.2074779458185], [16.240113401319277, 48.207461018233275], [16.240070399603866, 48.207476450654816], [16.239979393565573, 48.20736225057978], [16.23992731339159, 48.20729700614391], [16.239956555419386, 48.207286589477505], [16.239949562194138, 48.207277824227845], [16.239998352725177, 48.20726044516213], [16.240005332502225, 48.20726921039763], [16.240043904479418, 48.20725546502367], [16.240089877279292, 48.20731312301829], [16.240122226056116, 48.207353709473296]]]


In [441]:
layer = pdk.Layer(
    "GeoJsonLayer",
    data=specific_feature,
    get_fill_color=[255,0,0],#get_fill_color,  # Adjust color based on ANLAGENLEISTUNG
    get_line_color=[200, 0, 0],
    get_line_width=10,
    get_elevation="PV"
)

# Set the initial view state (adjust as needed)
view_state = pdk.ViewState(
    latitude=coordinates_adress[0],
    longitude=coordinates_adress[1],
    zoom=15,
)# Create a Pydeck deck
deck = pdk.Deck(layer, initial_view_state=view_state)
deck

{
  "initialViewState": {
    "latitude": 48.2024752,
    "longitude": 16.2784412,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": {
        "geometry": {
          "coordinates": [
            [
              [
                16.269742526005853,
                48.20154677577876
              ],
              [
                16.26970136056021,
                48.20154693186416
              ],
              [
                16.26970159705582,
                48.20157211304854
              ],
              [
                16.269535320634173,
                48.20157291617238
              ],
              [
                16.269534001124875,
                48.20144925847617
              ],
              [
                16.26967982896621,
                48.201448551455236
              ],
              [
                16.269696658358026,
                48.20144847124934
              ],
              [
                16.26970014261977,
                48.201448455276285
              ],
              [
                16.269700760136015,
                48.201491083550266
              ],
              [
                16.26972941440327,
                48.20149103715152
              ],
              [
                16.26973277758225,
                48.20149103010076
              ],
              [
                16.269741925420703,
                48.20149101739732
              ],
              [
                16.269742526005853,
                48.20154677577876
              ]
            ]
          ],
          "type": "Polygon"
        },
        "geometry_name": "SHAPE",
        "id": "SOLARANLAGLSTGOGD.36",
        "properties": {
          "ADRESSE": "14., Minorgasse 74",
          "ANLAGENLEISTUNG": 2,
          "BEZ": "1140",
          "GUT": 38,
          "MITTEL": 38,
          "OBJECTID": 36,
          "ONR": "74",
          "SCHLECHT": 15,
          "SEHRGUT": 0,
          "STR": "Minorgasse",
          "YR": 1041.6187741
        },
        "type": "Feature"
      },
      "getElevation": "@@=PV",
      "getFillColor": [
        255,
        0,
        0
      ],
      "getLineColor": [
        200,
        0,
        0
      ],
      "getLineWidth": 10,
      "id": "76bb9523-5b65-4b14-bab1-1967242af139"
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}